# 發行量加權股價指數歷史資料 爬蟲

#### 需要自訂的地方
1. 起始日期（股價跟成交量的日期要一樣）
2. csv_path 存成 csv 檔的檔案路徑
3. 資料庫連線設定

In [1]:
import requests
import pandas as pd
import time
import random
from datetime import datetime

## 開高低收資料下載

### 測試爬取單月

In [2]:
# # 設定爬蟲目標網址
# url = 'https://www.twse.com.tw/rwd/zh/TAIEX/MI_5MINS_HIST?date=20000101'

# # 發送HTTP請求
# response = requests.get(url)

# # 確認請求成功
# if response.status_code == 200:
#     # 將回應的文本轉換為JSON格式
#     data_json = response.json()
    
#     # 檢查狀態碼是否為'OK'
#     if data_json['stat'] == 'OK':
#         # 提取數據部分
#         data_entries = data_json['data']
        
#         # 轉換數據為DataFrame
#         df = pd.DataFrame(data_entries, columns=['date', 'open', 'high', 'low', 'close'])
        
#         # 處理日期格式，將民國年轉換為公元年並轉換為datetime型態
#         df['date'] = pd.to_datetime(df['date'].apply(lambda x: f"{1911+ int(x.strip().split('/')[0])}-{x.strip().split('/')[1]}-{x.strip().split('/')[2]}"))
        
#         # 顯示DataFrame
#         print(df)
#     else:
#         print("Data retrieval was not successful. Status:", data_json['stat'])
# else:
#     print("Failed to retrieve data, HTTP Status Code:", response.status_code)

### 爬取多月開高低收盤價

---

In [3]:
# 設定起始日期
# (年, 月, 日)
# 最早只有到1999年1月1日
start_date = datetime(2024, 10, 1)
start_date_vol = datetime(2024, 10, 1)

# 設定結束日期
end_date = datetime(2024, 12, 31)

---

In [4]:
all_data = pd.DataFrame()

# 設定爬蟲的起始和結束日期
start_date = start_date
current_date = datetime.now()

# 起始日期到當前日期的每個月份
while start_date <= current_date:

    formatted_date = start_date.strftime('%Y%m%d')

    url = f'https://www.twse.com.tw/rwd/zh/TAIEX/MI_5MINS_HIST?date={formatted_date}'
    
    # 加權報酬指數
    # url = f'https://www.twse.com.tw/rwd/zh/TAIEX/MFI94U?date={formatted_date}'

    response = requests.get(url)

    if response.status_code == 200:

        data_json = response.json()
        
        if data_json['stat'] == 'OK':

            data_entries = data_json['data']
            df = pd.DataFrame(data_entries, columns=['date', 'open', 'high', 'low', 'close'])
            
            # 處理日期格式，將民國年轉換為公元年並轉換成datetime
            df['date'] = pd.to_datetime(df['date'].apply(lambda x: f"{1911 + int(x.strip().split('/')[0])}-{x.strip().split('/')[1]}-{x.strip().split('/')[2]}"))

            df = df[df['date'] >= start_date]

            all_data = pd.concat([all_data, df], ignore_index=True)

            print(f"Data retrieved for {formatted_date}")

        elif data_json['stat'] == '查無資料':
            print(f"No data available for {formatted_date}")
            break  # 如果遇到"查無資料"則終止循環
    else:
        print(f"Failed to retrieve data for {formatted_date}, HTTP Status Code: {response.status_code}")

    # 將日期增加一個月
    next_month = start_date.month + 1 if start_date.month < 12 else 1
    next_year = start_date.year if next_month > 1 else start_date.year + 1
    start_date = datetime(next_year, next_month, 1)

    time.sleep(random.uniform(1, 3))

Data retrieved for 20241001
Data retrieved for 20241101
Data retrieved for 20241201


In [5]:
# print(all_data)

### 成交量

In [6]:
# volume_df = pd.DataFrame()

# start_date = start_date_vol
# current_date = datetime.now()

# while start_date <= current_date:

#     formatted_date = start_date.strftime('%Y%m%d')

#     url = f'https://www.twse.com.tw/exchangeReport/FMTQIK?date={formatted_date}'

#     try:
#         response = requests.get(url)

#         if response.status_code == 200:

#             data_json = response.json()
            
#             if data_json['stat'] == 'OK':

#                 data_entries = data_json['data']
                
#                 volume_data = [int(entry[1].replace(',', '')) for entry in data_entries]
#                 dates = [datetime.strptime(f"{1911 + int(entry[0].strip().split('/')[0])}-{entry[0].strip().split('/')[1]}-{entry[0].strip().split('/')[2]}", '%Y-%m-%d') for entry in data_entries]
                
#                 df = pd.DataFrame({'date': dates, 'volume': volume_data})

#                 df = df[df['date'] >= start_date]
                
#                 volume_df = pd.concat([volume_df, df], ignore_index=True)
                
#                 print(f"Data retrieved for {formatted_date}")

#             elif data_json['stat'] == '查無資料':
#                 print(f"No data available for {formatted_date}")
#                 break  # 如果遇到"查無資料"則終止循環
#         else:
#             print(f"Failed to retrieve data for {formatted_date}, HTTP Status Code: {response.status_code}")
#     except requests.exceptions.Timeout:
#         print("Request timed out for", formatted_date)
#     except requests.exceptions.RequestException as e:
#         print("Request failed:", e)

#     next_month = start_date.month + 1 if start_date.month < 12 else 1
#     next_year = start_date.year if next_month > 1 else start_date.year + 1
#     start_date = datetime(next_year, next_month, 1)

#     time.sleep(random.uniform(1.1, 3))


In [7]:
# print(volume_df)

In [8]:
# new_vol = (round(volume_df['volume']/1000,-3)).astype(int)

In [9]:
# all_data['volume'] = new_vol

In [10]:
all_data

,date,open,high,low,close
0,2024-10-01,"22,329.65","22,446.86","22,311.33","22,390.39"
1,2024-10-04,"22,459.10","22,493.05","22,193.50","22,302.71"
2,2024-10-07,"22,479.24","22,719.07","22,479.24","22,702.56"
3,2024-10-08,"22,691.32","22,691.32","22,418.38","22,611.39"
4,2024-10-09,"22,637.17","22,885.95","22,637.17","22,659.08"
...,...,...,...,...,...
57,2024-12-25,"23,226.35","23,296.36","23,159.36","23,220.13"
58,2024-12-26,"23,276.08","23,320.62","23,241.94","23,246.94"
59,2024-12-27,"23,251.28","23,335.66","23,204.35","23,275.68"
60,2024-12-30,"23,274.26","23,331.67","23,159.70","23,190.20"


## 儲存成 csv 檔

In [11]:
csv_path = f'../data/TW_data/taiex_data/taiex_data_{current_date.year}-{current_date.month}-{current_date.day}.csv'
all_data.to_csv(csv_path, index=False)

## 匯入資料庫

In [12]:
from sqlalchemy import create_engine
import configparser

In [13]:
# 資料庫連接設定
# 讀取 config.ini 檔案
config = configparser.ConfigParser()
config.read('../../DB_Processor/config.ini')

# 從 config.ini 中獲取資料庫連接設定
username = config['database']['user']
password = config['database']['password']
database = config['database']['db']
host = config['database']['host']
port = config['database']['port']

engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')
data = pd.read_csv(csv_path)
# data = pd.read_csv('../data/TW_data/taiex_data/taiex_data_2024-10-9.csv')

In [14]:
data

,date,open,high,low,close
0,2024-10-01,"22,329.65","22,446.86","22,311.33","22,390.39"
1,2024-10-04,"22,459.10","22,493.05","22,193.50","22,302.71"
2,2024-10-07,"22,479.24","22,719.07","22,479.24","22,702.56"
3,2024-10-08,"22,691.32","22,691.32","22,418.38","22,611.39"
4,2024-10-09,"22,637.17","22,885.95","22,637.17","22,659.08"
...,...,...,...,...,...
57,2024-12-25,"23,226.35","23,296.36","23,159.36","23,220.13"
58,2024-12-26,"23,276.08","23,320.62","23,241.94","23,246.94"
59,2024-12-27,"23,251.28","23,335.66","23,204.35","23,275.68"
60,2024-12-30,"23,274.26","23,331.67","23,159.70","23,190.20"


In [15]:
data['open'] = data['open'].replace(',', '', regex=True).astype(float)
data['high'] = data['high'].replace(',', '', regex=True).astype(float)
data['low'] = data['low'].replace(',', '', regex=True).astype(float)
data['close'] = data['close'].replace(',', '', regex=True).astype(float)
# data['volume'] = data['volume'].replace(',', '', regex=True).fillna(0).astype(int)
data['date'] = pd.to_datetime(data['date'])

In [16]:
# 只保留 date 在 end_date 之前的資料
data = data[data['date'] <= end_date]
# data = data[data['date'] <= '2024-09-30']
data = data[['date', 'open', 'high', 'low', 'close']]
# data = data[['date','close']]
data

,date,open,high,low,close
0,2024-10-01,22329.65,22446.86,22311.33,22390.39
1,2024-10-04,22459.10,22493.05,22193.50,22302.71
2,2024-10-07,22479.24,22719.07,22479.24,22702.56
3,2024-10-08,22691.32,22691.32,22418.38,22611.39
4,2024-10-09,22637.17,22885.95,22637.17,22659.08
...,...,...,...,...,...
57,2024-12-25,23226.35,23296.36,23159.36,23220.13
58,2024-12-26,23276.08,23320.62,23241.94,23246.94
59,2024-12-27,23251.28,23335.66,23204.35,23275.68
60,2024-12-30,23274.26,23331.67,23159.70,23190.20


### 將數據匯入到資料庫中的 taiex 資料表

In [17]:
data.to_sql('taiex', con=engine, if_exists='append', index=False)
# data.to_sql('adjustedtaiex', con=engine, if_exists='append', index=False)

62